# Cognitivo.ai
## Teste Desenvolvedor - Back End
### Rafael Alves Ribeiro - rafael.alves.ribeiro@gmail.com

## Cenário
<p style="text-align: justify;">Uma empresa que fornece aplicativos de música e livros, precisa acompanhar regularmente a evolução das métricas de aplicativos de música disponíveis na Apple Store. Hoje, para que esse acompanhamento seja feito, um profissional precisa diariamente realizar a coleta desses dados atualizados, realizar a transformação desses dados. Atualmente a empresa não dispõe de nenhuma ferramenta que faça esse trabalho de forma automatizada. Além disso, esse profissional está envolvido em diversas outras atividades, fazendo com que, muitas vezes, esses relatórios deixem de ser enviados.</p>

## Instruções
<p style="text-align: justify;">Utilizando a linguagem Python, acesse o arquivo AppleStore.csv que terá os seguintes dados disponíveis:
Dados disponíveis ​ id = Identificação do App track_name = Nome size_bytes = Tamanho em Bytes currency = Moeda price = Valor na Apple Store rating_count_tot = Qtde de Avaliaçõesrating_count_ver = Qtde de Avaliações última versão user_rating = Avaliação Média user_rating_ver = Avaliação Média da última versão ver = Última Versão cont_rating = Classificação Indicativa prime_genre = Gênero do App</p>

In [ ]:
%matplotlib inline

import datetime
import json
import os
import sys
import time

import pandas as pd
from sqlalchemy import create_engine
import tweepy
from tqdm import tqdm, tqdm_notebook

sys.path.append("..")

In [ ]:
from src import db, twitter, utils

### Carregamento dos dados

In [ ]:
CSV_FILE = "../dados/input/AppleStore.csv"
apple_store = pd.read_csv(CSV_FILE)
apple_store.drop("Unnamed: 0", axis=1, inplace=True)
apple_store.head()

<p style="text-align: justify;">Para isso, é preciso extrair os dados relativos a Aplicações “Apps”, do gênero News. Já para a Aplicação da categoria News, que tiver a maior quantidade de avaliações.

In [ ]:
news_top = apple_store.query('prime_genre == "News"').nlargest(1, "rating_count_tot")
print(
    f"Aplicação da categoria News com maior quantidade de avaliações: {news_top.track_name.iloc[0]}"
)

<p style="text-align: justify;">Utilizar a API dessa Aplicação para identificar quais são as 10 Aplicações do gênero Music e Book que possuem o maior número de citações.<p>

### Seleciona os apps dos gêneros Music e Book

* Criamos os datasets com os públicos Music e Book

In [ ]:
music_genre = apple_store.query('prime_genre == "Music"').reset_index(drop=True)
book_genre = apple_store.query('prime_genre == "Book"').reset_index(drop=True)
print(f"Music Apps: {len(music_genre)}")
print(f"Book Apps: {len(book_genre)}")

* Criamos o texto da query que será utilizado para pesquisar a API

In [ ]:
book_genre["api_query"] = book_genre.track_name.map(utils.quote)
book_genre[["track_name", "api_query"]].head()

In [ ]:
music_genre["api_query"] = music_genre.track_name.map(utils.quote)
music_genre[["track_name", "api_query"]].head()

### Busca as citações na API Twitter

In [ ]:
credentials = utils.load_credentials("../credentials.json")
api_credentials = credentials["twitter"]

counter = twitter.CitationsCounter(
    api_credentials["consumer_key"], api_credentials["consumer_secret"]
)

print("Rate limit:", counter.remaining_rate_limit)
print("Renew time:", counter.rate_limit_renew)

In [ ]:
tqdm.pandas(tqdm_notebook)  # adiciona barra de progresso ao map do Pandas

In [ ]:
book_genre["n_citacoes"] = book_genre.api_query.progress_map(counter.count_citations)

In [ ]:
music_genre["n_citacoes"] = music_genre.api_query.progress_map(counter.count_citations)

### Prepara os dados para armazenamento

In [ ]:
COLS = ["id", "track_name", "n_citacoes", "size_bytes", "price", "prime_genre"]
top10_book = book_genre.nlargest(10, "n_citacoes")
top10_book = top10_book[COLS].reset_index(drop=True)
top10_book

In [ ]:
COLS = ["id", "track_name", "n_citacoes", "size_bytes", "price", "prime_genre"]
top10_music = music_genre.nlargest(10, "n_citacoes")
top10_music = top10_music[COLS].reset_index(drop=True)
top10_music.head(10)

In [ ]:
dataset = pd.concat([top10_book, top10_music])

### Armazena no formato .csv

In [ ]:
outfile = f"citacoes_music_book_{datetime.date.today()}.csv"
dataset.to_csv(os.path.join("../dados/output", outfile), index=False)

Armazena os dados do arquivo criado no passo anterior em uma base de dados PostgreSQL criada na AWS RDS.

### Armazena em banco de dados (AWS RDS - Postgresql)

In [ ]:
db_credentials = credentials["db"]
db.upload_data_to_db(dataset, db_credentials["user"], db_credentials["password"])

### Disponibiliza os dados em formato JSON

Para acesso aos dados em formato JSON, criamos um app no Heroku que busca as informações no banco de dados hospedado na AWS e retorna em formato JSON:

Repositório: https://github.com/rafpyprog/topmobileapps


Aplicação: https://topmobileapps.herokuapp.com/